In [48]:
import pandas as pd
import numpy as np
import recordlinkage
from recordlinkage.preprocessing import clean, phonetic
from recordlinkage.datasets import load_febrl4
import numexpr as ne
import jellyfish

import warnings
warnings.filterwarnings("ignore")

In [50]:
pwd

'C:\\Users\\imoge'

The Freely Extensible Biomedical Record Linkage (Febrl) package is distributed with a dataset generator and four datasets generated with the generator. This function returns the fourth Febrl dataset as a pandas.DataFrame.

“Generated as one data set with 10000 records (5000 originals and 5000 duplicates, with one duplicate per original), the originals have been split from the duplicates, into dataset4a.csv (containing the 5000 original records) and dataset4b.csv (containing the 5000 duplicate records) These two data sets can be used for testing linkage procedures.”

In [2]:
dfA, dfB = load_febrl4()

In [3]:
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


In [4]:
dfA.shape

(5000, 10)

In [5]:
dfA.isnull().sum()

given_name       112
surname           48
street_number    158
address_1         98
address_2        420
suburb            55
postcode           0
state             50
date_of_birth     94
soc_sec_id         0
dtype: int64

In [6]:
dfB.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


In [7]:
dfB.shape

(5000, 10)

In [8]:
dfB.isnull().sum()

given_name       234
surname          102
street_number    287
address_1        220
address_2        851
suburb           106
postcode           0
state            107
date_of_birth    199
soc_sec_id         0
dtype: int64

# Record Pairs

In [9]:
# Basic get all record pairs
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(dfA, dfB)

In [10]:
len(pairs)

25000000

# Compare fields

In [23]:
#dfA, dfB = load_febrl4()

# Indexation step
indexer = recordlinkage.Index()
indexer.block('given_name')
candidate_links = indexer.index(dfA, dfB)

# Comparison step with parameters for each feature
compare_cl = recordlinkage.Compare()

compare_cl.exact('given_name', 'given_name', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, dfA, dfB)

# Classification step - more than 3 features matching
matches = features[features.sum(axis=1) > 3]
print(len(matches))

3241


# Deduplication

In [24]:
from recordlinkage.datasets import load_febrl1

In [25]:
dfA = load_febrl1()
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933


In [26]:
# Indexing - find links in the single dataframe
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(dfA)

In [27]:
len(candidate_links)

499500

In [35]:
# Get matches on first name
indexer = recordlinkage.Index()
indexer.block('given_name')
pairs = indexer.index(dfA)

print (len(pairs))

2082


In [36]:
compare_cl = recordlinkage.Compare()

compare_cl.exact('given_name', 'given_name', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(pairs, dfA)

In [37]:
features.head()

given_name  surname  date_of_birth  suburb  \
rec_id_1      rec_id_2                                                    
rec-183-dup-0 rec-122-org             1      0.0              0       0   
rec-248-org   rec-122-org             1      0.0              0       0   
              rec-183-dup-0           1      0.0              0       0   
rec-122-dup-0 rec-122-org             1      1.0              1       1   
              rec-183-dup-0           1      0.0              0       0   

                             state  address_1  
rec_id_1      rec_id_2                         
rec-183-dup-0 rec-122-org        0        0.0  
rec-248-org   rec-122-org        1        0.0  
              rec-183-dup-0      0        0.0  
rec-122-dup-0 rec-122-org        1        1.0  
              rec-183-dup-0      0        0.0

In [38]:
features.describe()

,given_name,surname,date_of_birth,suburb,state,address_1
count,2082.0,2082.000000,2082.000000,2082.000000,2082.000000,2082.000000
mean,1.0,0.144092,0.139289,0.108549,0.327089,0.133045
std,0.0,0.351268,0.346331,0.311148,0.469263,0.339705
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.0,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.0,1.000000,1.000000,1.000000,1.000000,1.000000


In [39]:
# Sum the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.0     142
5.0     145
4.0      30
3.0       9
2.0     376
1.0    1380
dtype: int64

In [40]:
matches = features[features.sum(axis=1) > 3]

print(len(matches))
matches.head(10)

317


,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-122-dup-0,rec-122-org,1,1.0,1,1,1,1.0
rec-183-org,rec-183-dup-0,1,1.0,1,1,1,1.0
rec-248-dup-0,rec-248-org,1,1.0,1,1,1,1.0
rec-373-dup-0,rec-373-org,1,1.0,1,1,1,1.0
rec-10-org,rec-10-dup-0,1,1.0,1,1,1,1.0
rec-342-dup-0,rec-342-org,1,1.0,0,1,1,1.0
rec-397-org,rec-397-dup-0,1,1.0,1,1,1,0.0
rec-472-org,rec-472-dup-0,1,1.0,1,1,1,0.0
rec-330-org,rec-330-dup-0,1,0.0,1,1,1,0.0


# Preprocessing

## Clean

recordlinkage.preprocessing.clean(s, lowercase=True, replace_by_none='[^ \\-\\_A-Za-z0-9]+', replace_by_whitespace='[\\-\\_]', strip_accents=None, remove_brackets=True, encoding='utf-8', decode_error='strict')

In [49]:
names = ['Mary-ann',
            'Bob :)',
            'Angel',
            'Bob (alias Billy)',
            None]
s = pd.Series(names)
print(clean(s))

0    mary ann
1         bob
2       angel
3         bob
4        None
dtype: object


* Phone numbers - remove all non-numbers except +
* Value occurrence - count number of times a value appears for each row

## Phonetic Encoding

* Indexing words by pronounciation
* recordlinkage.preprocessing.phonetic(s, method, concat=True, encoding='utf-8', decode_error='strict')
* preprocessing.phonetic_algorithms= ['soundex', 'nysiis', 'metaphone', 'match_rating']